In [1]:
import pandas as pd

In [2]:
#讀excel
df_orderData = pd.read_excel('訂單test.xlsx', sheet_name = 0)
df_orderData1 = pd.read_excel('訂單test1.xlsx', sheet_name = 0)

In [3]:
df = pd.DataFrame(df_orderData,columns=['OEB01','OEB03','OEB15','OEB16','OEA02','OEA14'])
df

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14
0,TT531-1804050001,1,05-APR-18,05-APR-18,05-APR-18,T0412
1,TT531-1804050001,2,05-APR-18,05-APR-18,05-APR-18,T0412
2,TT531-1804050001,3,05-APR-18,05-APR-18,05-APR-18,T0412
3,TT531-1804050001,4,05-APR-18,05-APR-18,05-APR-18,T0412
4,TT531-1804050001,5,05-APR-18,05-APR-18,05-APR-18,T0412
...,...,...,...,...,...,...
426,TT531-1804160001,1,14-MAY-18,22-MAY-18,16-APR-18,T0215
427,TT531-1804270001,1,13-JUN-18,11-JUN-18,27-APR-18,T0437
428,TT531-1804270001,2,13-JUN-18,15-JUN-18,27-APR-18,T0437
429,TT531-1804270001,3,13-JUN-18,30-MAY-18,27-APR-18,T0437


In [4]:
df["pk"] = df["OEB01"] + "_" + df["OEB03"].map(str)
df

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_2
2,TT531-1804050001,3,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_3
3,TT531-1804050001,4,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_4
4,TT531-1804050001,5,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_5
...,...,...,...,...,...,...,...
426,TT531-1804160001,1,14-MAY-18,22-MAY-18,16-APR-18,T0215,TT531-1804160001_1
427,TT531-1804270001,1,13-JUN-18,11-JUN-18,27-APR-18,T0437,TT531-1804270001_1
428,TT531-1804270001,2,13-JUN-18,15-JUN-18,27-APR-18,T0437,TT531-1804270001_2
429,TT531-1804270001,3,13-JUN-18,30-MAY-18,27-APR-18,T0437,TT531-1804270001_3


In [5]:
df1 = pd.DataFrame(df_orderData1,columns=['OEB01','OEB03','OEB15','OEB16','OEA02','OEA14'])
df1

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14
0,TT531-1804050001,1,05-APR-18,07-MAY-18,05-APR-18,T0412
1,TT531-1804050001,2,05-APR-18,07-MAY-18,05-APR-18,T0412
2,TT531-1804050001,3,05-APR-18,05-APR-18,05-APR-18,T0412
3,TT531-1804050001,4,05-APR-18,05-APR-18,05-APR-18,T0412
4,TT531-1804050001,5,05-APR-18,05-APR-18,05-APR-18,T0412
...,...,...,...,...,...,...
388,TT531-1804160001,1,14-MAY-18,22-MAY-18,16-APR-18,T0215
389,TT531-1804270001,1,13-JUN-18,11-JUN-18,27-APR-18,T0437
390,TT531-1804270001,2,13-JUN-18,15-JUN-18,27-APR-18,T0437
391,TT531-1804270001,3,13-JUN-18,30-MAY-18,27-APR-18,T0437


In [6]:
df1["pk"] = df1["OEB01"] + "_" + df1["OEB03"].map(str)
df1

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,07-MAY-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,07-MAY-18,05-APR-18,T0412,TT531-1804050001_2
2,TT531-1804050001,3,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_3
3,TT531-1804050001,4,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_4
4,TT531-1804050001,5,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_5
...,...,...,...,...,...,...,...
388,TT531-1804160001,1,14-MAY-18,22-MAY-18,16-APR-18,T0215,TT531-1804160001_1
389,TT531-1804270001,1,13-JUN-18,11-JUN-18,27-APR-18,T0437,TT531-1804270001_1
390,TT531-1804270001,2,13-JUN-18,15-JUN-18,27-APR-18,T0437,TT531-1804270001_2
391,TT531-1804270001,3,13-JUN-18,30-MAY-18,27-APR-18,T0437,TT531-1804270001_3


In [7]:
tmp = df.append(df1)
tmp.drop_duplicates(keep=False, inplace=True)
all_diff=pd.DataFrame(tmp)
all_diff

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_2
80,TT531-1804010001,1,31-MAY-18,22-MAY-18,01-APR-18,T0437,TT531-1804010001_1
81,TT531-1804010001,2,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_2
82,TT531-1804010001,3,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_3
83,TT531-1804010001,4,31-MAY-18,29-MAY-18,01-APR-18,T0437,TT531-1804010001_4
84,TT531-1804010001,5,31-MAY-18,01-JUN-18,01-APR-18,T0437,TT531-1804010001_5
85,TT531-1804010001,6,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_6
86,TT531-1804010001,7,31-MAY-18,31-MAY-18,01-APR-18,T0437,TT531-1804010001_7
87,TT531-1804010001,8,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_8


In [8]:
last = all_diff.drop_duplicates(subset=['pk'],keep='last')
last

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
80,TT531-1804010001,1,31-MAY-18,22-MAY-18,01-APR-18,T0437,TT531-1804010001_1
81,TT531-1804010001,2,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_2
82,TT531-1804010001,3,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_3
83,TT531-1804010001,4,31-MAY-18,29-MAY-18,01-APR-18,T0437,TT531-1804010001_4
84,TT531-1804010001,5,31-MAY-18,01-JUN-18,01-APR-18,T0437,TT531-1804010001_5
85,TT531-1804010001,6,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_6
86,TT531-1804010001,7,31-MAY-18,31-MAY-18,01-APR-18,T0437,TT531-1804010001_7
87,TT531-1804010001,8,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_8
88,TT531-1804010001,9,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_9
89,TT531-1804010001,10,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_10


In [9]:
first = all_diff.drop_duplicates(subset=['pk'],keep='first')
first

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_2
80,TT531-1804010001,1,31-MAY-18,22-MAY-18,01-APR-18,T0437,TT531-1804010001_1
81,TT531-1804010001,2,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_2
82,TT531-1804010001,3,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_3
83,TT531-1804010001,4,31-MAY-18,29-MAY-18,01-APR-18,T0437,TT531-1804010001_4
84,TT531-1804010001,5,31-MAY-18,01-JUN-18,01-APR-18,T0437,TT531-1804010001_5
85,TT531-1804010001,6,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_6
86,TT531-1804010001,7,31-MAY-18,31-MAY-18,01-APR-18,T0437,TT531-1804010001_7
87,TT531-1804010001,8,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_8


In [10]:
pk_diff = all_diff.drop_duplicates(subset=['pk'],keep=False)
pk_diff

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
80,TT531-1804010001,1,31-MAY-18,22-MAY-18,01-APR-18,T0437,TT531-1804010001_1
81,TT531-1804010001,2,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_2
82,TT531-1804010001,3,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_3
83,TT531-1804010001,4,31-MAY-18,29-MAY-18,01-APR-18,T0437,TT531-1804010001_4
84,TT531-1804010001,5,31-MAY-18,01-JUN-18,01-APR-18,T0437,TT531-1804010001_5
85,TT531-1804010001,6,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_6
86,TT531-1804010001,7,31-MAY-18,31-MAY-18,01-APR-18,T0437,TT531-1804010001_7
87,TT531-1804010001,8,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_8
88,TT531-1804010001,9,31-MAY-18,21-MAY-18,01-APR-18,T0437,TT531-1804010001_9
89,TT531-1804010001,10,31-MAY-18,14-MAY-18,01-APR-18,T0437,TT531-1804010001_10


In [11]:
new = last.append(pk_diff).drop_duplicates(subset=['pk'],keep=False)
new

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,07-MAY-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,07-MAY-18,05-APR-18,T0412,TT531-1804050001_2


In [12]:
old = first.append(pk_diff).drop_duplicates(subset=['pk'],keep=False)
old

,OEB01,OEB03,OEB15,OEB16,OEA02,OEA14,pk
0,TT531-1804050001,1,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_1
1,TT531-1804050001,2,05-APR-18,05-APR-18,05-APR-18,T0412,TT531-1804050001_2


In [13]:
final = new.merge(old,on = ['pk','OEB01','OEB03','OEB15','OEA02','OEA14'], suffixes=('_new','_old') )
final = final[['OEA14','OEA02','OEB01','OEB03','OEB15','OEB16_old','OEB16_new']]
final.sort_values('OEA14')
final.rename(columns={'OEA02': '訂單日期', 'OEA14': '業務編號', 'OEB01': '訂單號碼', 'OEB03': '項次', 'OEB15': '約定交貨日', 'OEB16_old': '原排定交貨日', 'OEB16_new': '新排定交貨日'}, inplace=True)
final

,業務編號,訂單日期,訂單號碼,項次,約定交貨日,原排定交貨日,新排定交貨日
0,T0412,05-APR-18,TT531-1804050001,1,05-APR-18,05-APR-18,07-MAY-18
1,T0412,05-APR-18,TT531-1804050001,2,05-APR-18,05-APR-18,07-MAY-18


In [14]:
#dataframe轉html
df_html = final.to_html(escape=False,index=False, justify = 'center')

In [15]:
def read_file(file):
    with open(file,'r',encoding="utf-8") as f:
            content_list = f.read().split('\n')
    return content_list
emails = read_file('email.txt')
emails

['hl94vul3h6@gmail.com',
 'eeecnzvlhavhksyb',
 'ytluo693@gmail.com',
 '----使用說明--------------------------------------------------------',
 '在第1行輸入寄件人Gmail帳號',
 '在第2行輸入寄件人Gmail密碼',
 '在第3行輸入收件人信箱，格式如下:',
 '    單個收件人: Fene1977@dayrep.com',
 '    多個收件人: Hinte1969@jourrapide.com, Fene1977@rhyta.com, Fene1977@teleworm.us',
 '注意: 檔名email.txt請勿更改。前三行請勿任意換行。',
 '--------------------------------------------------------------------']

In [16]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText#專門傳送正文
from email.mime.image import MIMEImage
import sys
import smtplib
import datetime

def SendMail():
    #print('start to send mail...')
    emails = read_file('email.txt') #讀email資料
    sender = emails[0] #step1:setup sender gmail,ex:"Fene1977@superrito.com"
    password = emails[1] #step2:setup sender gmail password
    recipients = emails[2] #step3:setup recipients mail
    today = datetime.date.today()
    sub = today.strftime("%m/%d") + "訂單交期更改通知" #step4:setup your subject
    
    #單個收件人,ex:recipients = 'Fene1977@dayrep.com'  
    #多個收件人,ex:"Hinte1969@jourrapide.com,Fene1977@rhyta.com,Fene1977@teleworm.us"
    
    outer = MIMEMultipart()
    outer['From'] = sender #step:setup sender gmail
    outer['To'] = recipients #step:setup recipient mail
    #outer["Cc"]="cc mail" #step:setup cc mail
    outer['Subject'] = sub #step:setup your subject

    #設定純文字資訊
    plainText = "偵測到訂單排定交貨日更改，請確認以下訂單："
    msgText = MIMEText(plainText, 'plain', 'utf-8')
    outer.attach(msgText)
    #設定HTML資訊
    htmlText = df_html #step7:edit your mail content
    msgText = MIMEText(htmlText, 'html', 'utf-8')
    outer.attach(msgText)

    mailBody = outer.as_string()
    #-----------------------------------------------------------------------
    
    # 寄送EMAIL
    try:
        with smtplib.SMTP(host="smtp.gmail.com", port="587") as s: #send webservice to gmail smtp socket
            s.ehlo()  # 驗證SMTP伺服器
            s.starttls()  # 建立加密傳輸
            s.login(sender, password)  # 登入寄件者gmail
            s.sendmail(sender, recipients,mailBody)  # 寄送郵件
            s.close()
        print("Email sent!")
    except:
        print("Unable to send the email. Error: ", sys.exc_info()[0])
        raise
        
if old.equals(new):
    print("未更改")
else:
    SendMail()
    print("交期已被更改")

Email sent!
交期已被更改
